In [11]:
import pickle
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)

with open('results.pkl', 'rb') as f:
    res = pickle.load(f)
print(res[0]['param'].keys())

df = pd.DataFrame([[str(np.array(res[data]['param']['X'], dtype=int)),
                        res[data]['param']['DIGESTMOD'],
                        res[data]['param']['QUALITY'],
                        res[data]['param']['PAYLOAD_SIZE_BYTE'],
                        res[data]['param']['ATTACK_PROBABILITY'],
                        np.nanmean(res[data]['goodput']), 
                        np.nanmean(res[data]['avg_verification']), 
                        np.nanmean(res[data]['latency']) - res[data]['param']['OFFSET'], 
                        np.nanmean(res[data]['frame_rate']) ] for data in res.keys()]
                        , columns=['X', 'DIGESTMOD', 'QUALITY', 'PAYLOAD_SIZE_BYTE', 'ATTACK_PROBABILITY', 'goodput', 'avg_verification', 'latency', 'frame_rate'])


dict_keys(['X', 'Y', 'KEY', 'DIGESTMOD', 'PAYLOAD_SIZE_BYTE', 'QUALITY', 'ATTACK_PROBABILITY', 'DURATION', 'OFFSET'])


/tmp/ipykernel_28306/121553894.py:18: RuntimeWarning: Mean of empty slice
  np.nanmean(res[data]['latency']) - res[data]['param']['OFFSET'],


In [12]:
df['X'] = pd.factorize(df['X'])[0]
df.groupby(['X', 'DIGESTMOD', 'QUALITY', 'PAYLOAD_SIZE_BYTE', 'ATTACK_PROBABILITY']).mean()

goodput  \
X DIGESTMOD QUALITY PAYLOAD_SIZE_BYTE ATTACK_PROBABILITY             
0 sha384    70      1024              0.00                0.868155   
1 sha384    25      64                0.00                0.510436   
                                      0.05                0.510086   
                                      0.10                0.464220   
                                      0.20                0.285457   
                                      0.30                0.125896   
                    128               0.00                0.633171   
                                      0.05                0.653255   
                                      0.10                0.576162   
                                      0.20                0.371533   
                                      0.30                0.159583   
                    1024              0.00                0.809774   
                                      0.05                0.820774   
                                      0.10                0.750357   
                                      0.20                0.501004   
                                      0.30                0.226928   
            75      64                0.00                0.521067   
                                      0.05                0.515971   
                                      0.10                0.467298   
                                      0.20                0.286639   
                                      0.30                0.126815   
                    128               0.00                0.666903   
                                      0.05                0.655768   
                                      0.10                0.594443   
                                      0.20                0.362197   
                                      0.30                0.163197   
                    1024              0.00                0.844933   
                                      0.05                0.813326   
                                      0.10                0.733520   
                                      0.20                0.480388   
                                      0.30                0.227018   
  sha512    25      64                0.00                0.437457   
                                      0.05                0.444753   
                                      0.10                0.406952   
                                      0.20                0.251901   
                                      0.30                0.108352   
                    128               0.00                0.575204   
                                      0.05                0.600262   
                                      0.10                0.542040   
                                      0.20                0.334175   
                                      0.30                0.148945   
                    1024              0.00                0.779003   
                                      0.05                0.786070   
                                      0.10                0.754500   
                                      0.20                0.493262   
                                      0.30                0.213294   
            75      64                0.00                0.457296   
                                      0.05                0.454292   
                                      0.10                0.408393   
                                      0.20                0.252430   
                                      0.30                0.110538   
                    128               0.00                0.613232   
                                      0.05                0.603509   
                                      0.10                0.546363   
                                      0.20                0.337325   
                                      0.30                0.155169   